<a href="https://colab.research.google.com/github/halzihaozhou/MLOPS-zoomcamp/blob/main/03_orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -qq prefect  mlflow pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0

In [ ]:
import prefect
prefect.__version__

'3.4.5'

In [ ]:
from google.colab import userdata

# token = userdata.get("ngrok")
# !ngrok config add-authtoken {token}

In [ ]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import mlflow

In [ ]:
# mlflow server \
#   --backend-store-uri sqlite:///mlflow.db \
#   --default-artifact-root file:/content/drive/MyDrive/MLOPS-zoomcamp/mlruns \
#   --host 127.0.0.1 \
#   --port 5000


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(f"MLflow UI with Registry: {public_url}")

MLflow UI with Registry: NgrokTunnel: "https://e8d1-35-245-149-150.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
mlflow.set_tracking_uri("https://10ae-35-236-203-218.ngrok-free.app")


In [ ]:
mlflow.set_experiment('Linear-regression-ny-march-yellow')
mlflow.sklearn.autolog()

2025/06/09 02:26:05 INFO mlflow.tracking.fluent: Experiment with name 'Linear-regression-ny-march-yellow' does not exist. Creating a new experiment.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/MLOPS-zoomcamp'

In [ ]:
df = pd.read_parquet(f'{folder_path}/yellow_tripdata_2023-03.parquet')

In [ ]:
print(len(df))

3403766


In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


In [ ]:
read_dataframe(f'{folder_path}/yellow_tripdata_2023-03.parquet')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.60,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.70,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.40,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.60,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.20,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403761,2,2023-03-31 23:24:25,2023-03-31 23:40:54,NaN,3.16,NaN,None,163,75,0,12.13,0.0,0.5,4.23,0.0,1.0,20.36,NaN,NaN,16.483333
3403762,2,2023-03-31 23:24:50,2023-04-01 00:04:12,NaN,6.89,NaN,None,125,198,0,40.92,0.0,0.5,8.98,0.0,1.0,53.90,NaN,NaN,39.366667
3403763,2,2023-03-31 23:26:31,2023-03-31 23:49:39,NaN,4.01,NaN,None,50,224,0,24.02,0.0,0.5,0.00,0.0,1.0,28.02,NaN,NaN,23.133333
3403764,2,2023-03-31 23:07:51,2023-03-31 23:15:56,NaN,1.31,NaN,None,113,158,0,8.51,0.0,0.5,3.50,0.0,1.0,16.01,NaN,NaN,8.083333


In [ ]:
def process(df):
    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)
    y_train = df["duration"].values

    return X_train, y_train, dv

In [ ]:
def train_best_model(X_train,y_train,dv):
    with mlflow.start_run():
        lr = LinearRegression()
        lr.fit(x_train, y_train)

In [32]:


!python {folder_path}/orchestrate.py

04:02:13.520 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8404
See https://docs.prefect.io/3.0/manage/self-host#self-host-a-prefect-server for more information on running a dedicated Prefect server.
04:02:19.473 | INFO    | Flow run 'scrupulous-kagu' - Beginning flow run 'scrupulous-kagu' for flow 'main-flow'
04:02:27.487 | INFO    | Task run 'read_dataframe-cdc' - Finished in state Completed()
04:02:50.725 | INFO    | Task run 'process-609' - Finished in state Completed()
2025/06/09 04:04:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
🏃 View run fortunate-skink-197 at: https://10ae-35-236-203-218.ngrok-free.app/#/experiments/5/runs/2d73186703c046f68543eb92e3054f3a
🧪 View experiment at: https://10ae-35-236-203-218.ngrok-free.app/#/experiments/5
04:04:15.228 | INFO    | Task run 'train_best_model-24e' - Finished in state Completed()
0

In [33]:
# View model
import pickle
with open("/content/drive/MyDrive/MLOPS-zoomcamp/mlruns/5/2d73186703c046f68543eb92e3054f3a/artifacts/model/model.b", "rb") as f_in:
    dv, lr = pickle.load(f_in)

print("Intercept:", lr.intercept_)
print("Coefficients:", lr.coef_)

Intercept: 24.77203445209766
Coefficients: [ 1.90326752e+01 -2.53335987e+01 -4.35578976e+00 -6.54128073e+00
 -1.18592898e+00  7.00941416e+00 -6.78880350e+00  2.45899332e+00
  1.75690065e+01  4.50621988e+00  6.04100760e+00  1.36437240e-01
 -5.31354532e+00 -4.73759583e+00  9.17404346e+00  1.49763156e+00
 -1.21099690e+00  9.44751924e+00  2.10204152e+00  9.69014639e-01
  2.74957373e+00 -6.39232418e+00 -8.10768419e+00  4.88968907e+00
 -1.62780732e+01 -4.16987330e+00  3.15522842e+00  4.40479026e+00
  5.88488083e+00 -1.58506344e+00 -4.19725125e-01 -1.15307613e+01
 -6.96754974e+00  1.66216056e+01  7.84731966e+00 -7.93519444e+00
 -8.76824512e+00  6.99193942e+00 -6.80976849e+00  8.57247647e+00
 -1.91402952e+01  8.70547417e+00 -5.00866653e+00 -6.49533330e+00
 -5.74798082e+00 -4.99896658e+00 -2.56259592e+00 -2.59937536e+00
 -2.22156972e+00  2.03252496e+00 -2.93807789e+00  2.12075985e+00
 -5.90916635e+00 -2.12032417e+00 -4.82992286e+00 -1.43041646e+00
  6.24173405e+00 -1.76808457e+01 -5.62698270e+0

In [39]:
import yaml
import os

mlmodel_path = "/content/drive/MyDrive/MLOPS-zoomcamp/mlruns/5/2d73186703c046f68543eb92e3054f3a/artifacts/sklearn-model/model.pkl"

size_bytes = os.path.getsize(mlmodel_path)

print(f"📦 model.pkl size: {size_bytes} bytes ")


📦 model.pkl size: 4501 bytes 
